In [ ]:
# Seems that the restart files do not quite line up with how
# I'm setting the tracer releases.
import os
from stompy.grid import unstructured_grid
import matplotlib.pyplot as plt
import xarray as xr
import local_config
%matplotlib notebook

In [ ]:
base=os.path.join(local_config.run_root_dir,"2018_Suisun_R05_res00")        

In [ ]:
# Load a partitioned grid
num_procs=16
g_ins=[]
for proc in range(num_procs):
    g_in_fn=os.path.join(base,
                         f"2017_suisun_marsh_72_VegCorr_D_{proc:04d}_net.nc")
    g_in=unstructured_grid.UnstructuredGrid.read_dfm(g_in_fn)
    g_ins.append(g_in)

In [ ]:
plt.figure()
for g_in in g_ins:
    g_in.plot_edges()

In [ ]:
# Now load a restart file
rst_in_fn=os.path.join(base,
                      "DFM_OUTPUT_FlowFM",
                      f"FlowFM_{proc:04d}_20180415_000000_rst.nc")
rst_in=xr.open_dataset(rst_in_fn)

In [ ]:
rst_outs=[]
for proc in range(num_procs):
    rst_out_fn=os.path.join(base,
                          "DFM_OUTPUT_FlowFM_R001",
                          f"FlowFM_{proc:04d}_20180410_000000_rst.nc")
    rst_out=xr.open_dataset(rst_out_fn)
    rst_outs.append(rst_out)

In [ ]:
# still, the grid should be okay.
plt.figure()

# Everybody looks correct.
for g_in,rst_out in zip(g_ins,rst_outs):
    g_in.plot_cells(values=rst_out['pt'].isel(time=0),
                   clim=[0,1])
plt.axis('equal')

In [ ]:
# Map output from previous run
# might match, but definitely requires that the reordering
# deterministic.
map_ins=[]
for proc in range(num_procs):
    map_in_fn=os.path.join(base,
                          "DFM_OUTPUT_FlowFM",
                          f"FlowFM_{proc:04d}_map.nc")
    map_ins.append( xr.open_dataset(map_in_fn) )


In [ ]:
# And compare with map out
map_outs=[]
for proc in range(num_procs):
    map_out_fn=os.path.join(base,
                          "DFM_OUTPUT_FlowFM_R001",
                          f"FlowFM_R001_{proc:04d}_map.nc")
    map_out=xr.open_dataset(map_out_fn)
    map_outs.append(map_out)

In [ ]:
map_gs=[ unstructured_grid.UnstructuredGrid.from_ugrid(ds)
        for ds in map_outs]


In [ ]:
for rst_out,g_in,map_g in zip(rst_outs,g_ins,map_gs):
    #print("Cells in restart rst_in:   ",rst_in.dims['nFlowElem'])
    print("Cells in partitioned grid: ",g_in.Ncells())
    print("Cells in restart rst_out:  ",rst_out.dims['nFlowElem'])
    print("Cell in map output grid:   ",map_g.Ncells())
    print()

In [ ]:
plt.figure()
for map_g in map_gs:
    map_g.plot_edges()
    

In [ ]:
# This shows that g_in and the grid in the map output are 
# different, and while they have the same number of cells
# the order is somehow different.  Thus 
fig,axs=plt.subplots(1,3,figsize=(9.5,8))

for g_in,map_g,rst_out,map_out in zip(g_ins,map_gs,rst_outs,map_outs):
    g_in.plot_cells(values=rst_out['pt'].isel(time=0),
                   clim=[0,1],ax=axs[0])
    map_g.plot_cells(values=rst_out['pt'].isel(time=0),
                   clim=[0,1],ax=axs[1])
    map_g.plot_cells(values=map_out['mesh2d_pt'].isel(time=0),
                     clim=[0,1],ax=axs[2])
    
axs[0].axis('equal')
axs[1].axis('equal')
axs[2].axis('equal')

In [ ]:
rst_outs[0].FlowElem_xzw

In [ ]:
map_outs[0].mesh2d_face_x

In [ ]:
plt.figure()
plt.plot( rst_outs[0].FlowElem_xzw,
          map_outs[0].mesh2d_face_x,
         'g.')
plt.plot( rst_outs[0].FlowElem_xzw,
          g_ins[0].cells_centroid()[:,0],
         'r.')
plt.plot( rst_outs[0].FlowElem_xzw,
          map_ins[0].mesh2d_face_x,
         'b.')

In [ ]:
# So there is some sort of renumbering that occurs between reading in a partitioned grid
# and the internal workings of DFM.
# currently I load a subdomain_grid to figure out how to write
# the IC.

In [ ]:
rst_outs[0]

In [ ]:
g_